# Seq2Seq模型实现文本翻译

参考论文：[Sequence to Sequence Learning with Neural Networks](https://arxiv.org/abs/1409.3215)

## 概述

## 数据准备

我们本次使用的数据集为**Multi30K数据集**，它是一个大规模的图像-文本数据集，包含30K+图片，每张图片对应两类不同的文本描述：
- 英语描述，及对应的德语翻译；
- 五个独立的、非翻译而来的英语和德语描述，描述中包含的细节并不相同；

因其收集的不同语言对于图片的描述相互独立，所以训练出的模型可以更好地适用于有噪声的多模态内容。

![avatar](./assets/Multi30K.png)

首先，我们通过如下url下载数据：

In [1]:
# 训练、验证、测试数据集下载地址
urls = {
    'train': 'http://www.quest.dcs.shef.ac.uk/wmt16_files_mmt/training.tar.gz',
    'valid': 'http://www.quest.dcs.shef.ac.uk/wmt16_files_mmt/validation.tar.gz',
    'test': 'http://www.quest.dcs.shef.ac.uk/wmt17_files_mmt/mmt_task1_test2016.tar.gz'
}

### 数据下载模块

数据下载模块使用`requests`库进行http请求，并通过`tqdm`对下载百分比进行可视化。此外针对下载安全性，使用IO的方式下载临时文件，而后保存至指定的路径并返回。
> `tqdm`和`requests`库需手动安装，命令如下：`pip install tqdm requests`

In [2]:
import os
import logging
import shutil
import requests
import tempfile
from tqdm import tqdm
from typing import IO
from pathlib import Path

# 指定保存路径为 `home_path/.mindspore_examples`
cache_dir = Path.home() / '.mindspore_examples'

def http_get(url: str, temp_file:IO):
    """使用requests库下载数据，并使用tqdm库进行流程可视化"""
    req = requests.get(url, stream=True)
    content_length = req.headers.get('Content-Length')
    total = int(content_length) if content_length is not None else None
    progress = tqdm(unit='B', total=total)
    for chunk in req.iter_content(chunk_size=1024):
        if chunk:
            progress.update(len(chunk))
            temp_file.write(chunk)
    progress.close()

def download(file_name:str, url: str):
    """下载数据并存为指定名称"""
    if not os.path.exists(cache_dir):
        os.makedirs(cache_dir)
    cache_path = os.path.join(cache_dir, file_name)
    cache_exist = os.path.exists(cache_path)
    if not cache_exist:
        with tempfile.NamedTemporaryFile() as temp_file:
            http_get(url, temp_file)
            temp_file.flush()
            temp_file.seek(0)
            logging.info(f"copying {temp_file.name} to cache at {cache_path}")
            with open(cache_path, 'wb') as cache_file:
                shutil.copyfileobj(temp_file, cache_file)
    return cache_path

下载后的训练、验证、测试数据集各对应一个`tar.gz`文件，以训练数据集为例，原始的解压目录如下：
```text
training
├── train.de
└── train.en
```

In [3]:
train_path = download('train.tar.gz', urls['train'])
valid_path = download('valid.tar.gz', urls['valid'])
test_path = download('test.tar.gz', urls['test'])

### 数据预处理

在使用数据进行模型训练等操作时，我们需要对数据进行预处理，流程如下：

1. 解压`tar.gz`文件；
2. 目前数据为句子形式的文本，需要进行分词，即将句子拆解为单独的词元（token，可以为字符或者单词）；
    - 分词可以使用`spaCy`创建分词器（tokenizer）：`de_core_news_sm`，`en_core_web_sm`，需要手动下载；
    - 分词后，去除多余的空格，统一大小写等；
3. 将每个词元映射到从0开始的数字索引中（为节约存储空间，可过滤掉词频低的词元），词元和数字索引所构成的集合叫做词典（vocabulary）；
4. 添加特殊占位符，标明序列的起始与结束，统一序列长度，并创建数据迭代器；

> `spaCy` 下载： `pip install spacy`

> 分词器（tokennizer）下载： `python -m spacy download de_core_news_sm`，`python -m spacy download en_core_web_sm`


#### 数据加载器（解压+分词）

In [4]:
import re
import six
import string
import tarfile
import spacy
from functools import partial

class Multi30K():
    """Multi30K数据集加载器
    
    加载Multi30K数据集并处理为一个Python迭代对象。
    
    """
    def __init__(self, path):
        self.data = self._load(path)
        
    def _load(self, path):
        def tokenize(text, spacy_lang):
            # 去除多余空格，统一大小写
            text = text.rstrip()
            return [tok.text.lower() for tok in spacy_lang.tokenizer(text)]
        
        # 加载英、德语分词器
        tokenize_de = partial(tokenize, spacy_lang=spacy.load('de_core_news_sm'))
        tokenize_en = partial(tokenize, spacy_lang=spacy.load('en_core_web_sm'))
        
        # 解压、读取Multi30K数据，并进行分词
        tarf = tarfile.open(path)
        members = {i.name.split('.')[-1]: i for i in tarf.getmembers()}
        de = tarf.extractfile(members['de']).readlines()[:-1]
        de = [tokenize_de(i.decode()) for i in de]
        en = tarf.extractfile(members['en']).readlines()[:-1]
        en = [tokenize_en(i.decode()) for i in en]

        return list(zip(de, en))
        
    def __getitem__(self, idx):
        return self.data[idx]
    
    def __len__(self):
        return len(self.data)

In [5]:
train_dataset, valid_dataset, test_dataset = Multi30K(train_path), Multi30K(valid_path), Multi30K(test_path)

对解压和分词结果进行测试，打印测试数据集第一组英德语文本，可以看到每一个单词和标点符号已经被单独分离出来。

In [6]:
for de, en in test_dataset:
    print(de, en)
    break

['ein', 'mann', 'mit', 'einem', 'orangefarbenen', 'hut', ',', 'der', 'etwas', 'anstarrt', '.'] ['a', 'man', 'in', 'an', 'orange', 'hat', 'starring', 'at', 'something', '.']


#### 词典

In [7]:
class Vocab:
    """通过词频字典，构建词典"""

    def __init__(self, word_count_dict, min_freq=1, special_tokens=['<unk>', '<pad>', '<bos>', '<eos>']):
        self.word2idx = {}
        for idx, tok in enumerate(special_tokens):
            self.word2idx[tok] = idx

        # 过滤低词频的词元
        filted_dict = {
            w: c
            for w, c in word_count_dict.items() if c >= min_freq
        }
        for w, _ in filted_dict.items():
            self.word2idx[w] = len(self.word2idx)

        self.idx2word = {idx: word for word, idx in self.word2idx.items()}

        self.bos_idx = self.word2idx['<bos>']  # 特殊占位符：序列开始
        self.eos_idx = self.word2idx['<eos>']  # 特殊占位符：序列结束
        self.pad_idx = self.word2idx['<pad>']  # 特殊占位符：补充字符
        self.unk_idx = self.word2idx['<unk>']  # 特殊占位符：低词频词元或未曾出现的词元

    def _word2idx(self, word):
        """单词映射至数字索引"""
        if word not in self.word2idx:
            return self.unk_idx
        return self.word2idx[word]

    def _idx2word(self, idx):
        """数字索引映射至单词"""
        if idx not in self.idx2word:
            raise ValueError('input index is not in vocabulary.')
        return self.idx2word[idx]

    def encode(self, word_or_list):
        """将单个单词或单词数组映射至单个数字索引或数字索引数组"""
        if isinstance(word_or_list, list):
            return [self._word2idx(i) for i in word_or_list]
        return self._word2idx(word_or_list)

    def decode(self, idx_or_list):
        """将单个数字索引或数字索引数组映射至单个单词或单词数组"""
        if isinstance(idx_or_list, list):
            return [self._idx2word(i) for i in idx_or_list]
        return self._idx2word(idx_or_list)

    def __len__(self):
        return len(self.word2idx)

通过自定义词频字典进行测试，我们可以看到词典已去除词频少于2的词元c，并加入了默认的四个特殊占位符，故词典整体长度为：4 - 1 + 4 = 7

In [8]:
word_count = {'a':20, 'b':10, 'c':1, 'd':2}

vocab = Vocab(word_count, min_freq=2)
len(vocab)

7

使用`collections`中的`Counter`和`OrderedDict`统计英/德语每个单词在整体文本中出现的频率。构建词频字典，然后再将词频字典转为词典。

在分配数字索引时有一个小技巧：常用的词元对应数值较小的索引，这样可以节约空间。

> `collections` 需要手动下载： `pip install collections`

In [9]:
from collections import Counter, OrderedDict

def build_vocab(dataset):
    de_words, en_words = [], []
    for de, en in dataset:
        de_words.extend(de)
        en_words.extend(en)
        
    de_count_dict = OrderedDict(sorted(Counter(de_words).items(), key=lambda t: t[1], reverse=True))
    en_count_dict = OrderedDict(sorted(Counter(en_words).items(), key=lambda t: t[1], reverse=True))
    
    return Vocab(de_count_dict, min_freq=2), Vocab(en_count_dict, min_freq=2)

In [10]:
de_vocab, en_vocab = build_vocab(train_dataset)
print('Unique tokens in de vocabulary:', len(de_vocab))

Unique tokens in de vocabulary: 7853


#### 数据迭代器

数据预处理的最后一步是创建数据迭代器，我们在进一步处理数据（包括批处理，添加起始和终止符号，统一序列长度）后，将数据以张量的形式返回。

创建数据迭代器需要如下参数：

- `dataset`：分词后的数据集
- `de_vocab`：德语词典
- `en_vocab`：英语词典
- `batch_size`：批量大小，即一个batch中包含多少个序列
- `max_len`：序列最大长度，为最长有效文本长度 + 2（序列开始、序列结束占位符），如不满则补齐，如超过则丢弃
- `drop_remainder`：是否在最后一个batch未满时，丢弃该batch

In [11]:
import mindspore

class Iterator():
    """创建数据迭代器"""
    def __init__(self, dataset, de_vocab, en_vocab, batch_size, max_len=32, drop_reminder=False):
        self.dataset = dataset
        self.de_vocab = de_vocab
        self.en_vocab = en_vocab
        
        self.batch_size = batch_size
        self.max_len = max_len
        self.drop_reminder = drop_reminder

        length = len(self.dataset) // batch_size 
        self.len = length if drop_reminder else length + 1 # 批量数量
    
    def __call__(self):
        def pad(idx_list, vocab, max_len):
            """统一序列长度，并记录有效长度"""
            idx_pad_list, idx_len = [], []
            # 当前序列度超过最大长度时，将超出的部分丢弃；当前序列长度小于最大长度时，用占位符补齐
            for i in idx_list:
                if len(i) > max_len - 2:
                    idx_pad_list.append(
                        [vocab.bos_idx] + i[:max_len-2] + [vocab.eos_idx]
                    )
                    idx_len.append(max_len)
                else:
                    idx_pad_list.append(
                        [vocab.bos_idx] + i + [vocab.eos_idx] + [vocab.pad_idx] * (max_len - len(i) - 2)
                    )
                    idx_len.append(len(i) + 2)
            return idx_pad_list, idx_len

        def sort_by_length(src, trg):
            """对德/英语的字段长度进行排序"""
            data = zip(src, trg)
            data = sorted(data, key=lambda t: len(t[0]), reverse=True)
            return zip(*list(data))
            
        def encode_and_pad(batch_data, max_len):
            """将批量中的文本数据转换为数字索引，并统一每个序列的长度"""
            # 将当前批量数据中的词元转化为索引
            src_data, trg_data = zip(*batch_data)
            src_idx = [self.de_vocab.encode(i) for i in src_data]
            trg_idx = [self.en_vocab.encode(i) for i in trg_data]
            
            # 统一序列长度
            src_idx, trg_idx = sort_by_length(src_idx, trg_idx)
            src_idx_pad, src_len = pad(src_idx, de_vocab, max_len)
            trg_idx_pad, _ = pad(trg_idx, en_vocab, max_len)
            
            return src_idx_pad, src_len, trg_idx_pad
        
        for i in range(self.len):
            # 获取当前批量的数据
            if i == self.len - 1 and not self.drop_reminder:
                batch_data = self.dataset[i * self.batch_size:]
            else:
                batch_data = self.dataset[i * self.batch_size: (i+1) * self.batch_size]
            
            src_idx, src_len, trg_idx = encode_and_pad(batch_data, self.max_len)
            # 将序列数据转换为tensor
            yield mindspore.Tensor(src_idx, mindspore.int32), \
                mindspore.Tensor(src_len, mindspore.int32), \
                mindspore.Tensor(trg_idx, mindspore.int32)
    
    def __len__(self):
        return self.len

In [12]:
train_iterator = Iterator(train_dataset, de_vocab, en_vocab, batch_size=128, max_len=32, drop_reminder=True)
valid_iterator = Iterator(valid_dataset, de_vocab, en_vocab, batch_size=128, max_len=32, drop_reminder=False)
test_iterator = Iterator(test_dataset, de_vocab, en_vocab, batch_size=128, max_len=32, drop_reminder=False)

## 模型构建

### 编码器（Encoder）

在编码器中，我们输入一个序列$X=\{x_1, x_2, ..., x_T\}$，在embedding层将其转化为向量，循环计算隐藏状态$H=\{h_1, h_2, ..., h_T\}$，并在最后的隐藏状态中返回上下文向量$z=h_T$。

实现编码器的方式有很多种，在这里我们使用的是门控循环单元模型（Gated Rrecurrent Units, GRU）。它在原始循环神经网络（Recurrent Neural Network，RNN）的基础上引入了门机制（gate mechanism），用以控制输入隐藏状态和从隐藏状态输出的信息。其中，更新门（update gate， 又称记忆门，一般用$z_t$表示）用于控制前一时刻的状态信息$h_{t-1}$被带入到当前状态$h_t$中的程度。重置门（reset gate，一般用$r_t$表示）控制前一状态$h_t$有多少信息被写入到当前候选集$n_t$上。

$$h_t = \text{RNN}(e(x_t), h_{t-1})$$

在进行文本翻译类任务时，我们一般使用双向GRU，即在训练中同时考虑当前词语之前及之后的文本内容。双向GRU的每层由两个RNN构成，前向RNN由左至右循环计算隐藏状态，反向RNN从右至左计算隐藏状态，公式表达如下：

$$\begin{align*}
h_t^\rightarrow &= \text{EncoderGRU}^\rightarrow(e(x_t^\rightarrow),h_{t-1}^\rightarrow)\\
h_t^\leftarrow &= \text{EncoderGRU}^\leftarrow(e(x_t^\leftarrow),h_{t-1}^\leftarrow)
\end{align*}$$

每个RNN网络在观察到句子中的最后一个词后，输出一个上下文向量，前向RNN的输出为$z^\rightarrow=h_T^\rightarrow$，反向RNN的输出为$z^\leftarrow=h_T^\leftarrow$。

![avatar](./assets/bidirectional-gru.png)

编码器最终会返回两项：`outputs`和`hidden`。

- `outputs`为双向GRU最上层隐藏状态，形状为\[max_len, batch_size, hid_dim * num_directions\]。以$t=1$时刻为例，其对应的output为前向RNN中$t=1$时刻最上层隐藏状态和反向RNN中$t=T$时刻的结合，即$h_1 = [h_1^\rightarrow; h_{T}^\leftarrow]$；

- `hidden`表示每层的最终隐藏状态，即上文提到的上下文向量。但由于编码器（decoder）的结构并不是双向的，仅仅需要一个上下文向量$z$，为了与之对应，我们将编码器中的两个向量组合起来，放入全连接层$g$中，并最后使用激活函数$tanh$；

$$z=\tanh(g(h_T^\rightarrow, h_T^\leftarrow)) = \tanh(g(z^\rightarrow, z^\leftarrow)) = s_0$$

MindSpore为大家提供了GRU的接口，可以在编码器搭建中直接调用，通过设置参数`bidirectional=True`使用双向GRU。

In [13]:
import mindspore
import mindspore.nn as nn
import mindspore.ops as ops
import mindspore.numpy as mnp

class Encoder(nn.Cell):
    def __init__(self, input_dim, emb_dim, enc_hid_dim, dec_hid_dim, dropout):
        super().__init__()
        self.embedding = nn.Embedding(input_dim, emb_dim) # Embedding层
        self.rnn = nn.GRU(emb_dim, enc_hid_dim, bidirectional=True).to_float(compute_dtype) # 双向GRU层
        self.fc = nn.Dense(enc_hid_dim * 2, dec_hid_dim).to_float(compute_dtype) # 全连接层

        self.dropout = nn.Dropout(1-dropout) # dropout，防止过拟合
        
    def construct(self, src, src_len):
        """构建编码器
        
        Args:
            src：源序列，为已经转换为数字索引并统一长度的序列；shape = [max_len, batch_size]
            src_len: 有效长度；shape = [batch_size, ]
        """

        # 将输入源序列转化为向量，并进行暂退（dropout），shape = [序列最大长度, batch size, emb dim]
        embedded = self.dropout(self.embedding(src))
        # 计算输出             
        outputs, hidden = self.rnn(embedded, seq_length=src_len)                       
        # 为适配解码器，合并两个上下文函数
        hidden = ops.tanh(self.fc(mnp.concatenate((hidden[-2,:,:], hidden[-1,:,:]), axis = 1)))
        
        return outputs, hidden

### 注意力层（Attention）

在机器翻译中，每个生成的词可能对应源句子中不同的词，而传统的无注意力机制的seq2seq模型更偏向于关注句子中的最后一个词。为了进一步优化模型，我们引入了注意力机制。

注意力机制便是赋予源句子和目标句子中对应的词以更高的权重，它整合了我们目前为止编码与解码的所有信息，并输出一个表示注意力权重的向量$a_t$，用来决定在下一步的预测$\hat{y}_t+1$中应该给予哪些词更高的关注度。

首先，我们需要明确编码器中的每一个隐藏状态和解码器中上一个时刻隐藏状态之间匹配的程度$E_t$。截止到当前的时刻$t$，编码器（encoder）中的所有信息为全部前向和后向RNN的隐藏状态的组合$H$，是一个有$T$个tensor的序列；解码器（decoder）中的所有信息为上一时刻的隐藏状态$s_{t-1}$，是一个单独的tensor。为了统一二者的维度，我们需要将解码器中上一时刻的隐藏状态$s_{t-1}$重复$T$次，接着把处理好的解码器信息与编码器信息堆叠起来，并输入到线性层`att`和激活函数$tanh$中，计算编码器与解码器隐藏状态之间的能量$E_t$。

$$E_t = \tanh(\text{attn}(s_{t-1}, H))$$ 

In [14]:
class Attention(nn.Cell):
    def __init__(self, enc_hid_dim, dec_hid_dim):
        super().__init__()
        
        self.attn = nn.Dense((enc_hid_dim * 2) + dec_hid_dim, dec_hid_dim).to_float(compute_dtype)
        self.v = nn.Dense(dec_hid_dim, 1, has_bias = False).to_float(compute_dtype)
        
    def construct(self, hidden, encoder_outputs, mask):
        
        #hidden = [batch size, dec hid dim]
        #encoder_outputs = [src len, batch size, enc hid dim * 2]
        
        batch_size = encoder_outputs.shape[1]
        src_len = encoder_outputs.shape[0]
        
        #repeat decoder hidden state src_len times
        hidden = mnp.tile(hidden.expand_dims(1), (1, src_len, 1))
  
        encoder_outputs = encoder_outputs.transpose(1, 0, 2)
        
        #hidden = [batch size, src len, dec hid dim]
        #encoder_outputs = [batch size, src len, enc hid dim * 2]
        
        energy = ops.tanh(self.attn(mnp.concatenate((hidden, encoder_outputs), axis = 2))) 
        
        #energy = [batch size, src len, dec hid dim]

        attention = self.v(energy).squeeze(2)
        
        #attention = [batch size, src len]
        
        attention = attention.masked_fill(mask == 0, -1e10)
        
        return ops.Softmax(1)(attention)

### 解码器（Decoder）

In [15]:
class Decoder(nn.Cell):
    def __init__(self, output_dim, emb_dim, enc_hid_dim, dec_hid_dim, dropout, attention):
        super().__init__()
        self.output_dim = output_dim
        self.attention = attention
        
        self.embedding = nn.Embedding(output_dim, emb_dim)
        self.rnn = nn.GRU((enc_hid_dim * 2) + emb_dim, dec_hid_dim).to_float(compute_dtype)
        self.fc_out = nn.Dense((enc_hid_dim * 2) + dec_hid_dim + emb_dim, output_dim).to_float(compute_dtype)
        self.dropout = nn.Dropout(1-dropout)
        
    def construct(self, inputs, hidden, encoder_outputs, mask):
             
        #input = [batch size]
        #hidden = [batch size, dec hid dim]
        #encoder_outputs = [src len, batch size, enc hid dim * 2]
        #mask = [batch size, src len]
        
        inputs = inputs.expand_dims(0)
        #input = [1, batch size]
        
        embedded = self.dropout(self.embedding(inputs))
        
        #embedded = [1, batch size, emb dim]
        a = self.attention(hidden, encoder_outputs, mask)
                
        #a = [batch size, src len]
        
        a = a.expand_dims(1)
        
        #a = [batch size, 1, src len]
        
        encoder_outputs = encoder_outputs.transpose(1, 0, 2)
        
        #encoder_outputs = [batch size, src len, enc hid dim * 2]
        
        weighted = ops.BatchMatMul()(a, encoder_outputs)
        
        #weighted = [batch size, 1, enc hid dim * 2]
        
        weighted = weighted.transpose(1, 0, 2)
        
        #weighted = [1, batch size, enc hid dim * 2]
        
        rnn_input = mnp.concatenate((embedded, weighted), axis = 2)
        
        #rnn_input = [1, batch size, (enc hid dim * 2) + emb dim]
            
        output, hidden = self.rnn(rnn_input, hidden.expand_dims(0))
        
        #output = [seq len, batch size, dec hid dim * n directions]
        #hidden = [n layers * n directions, batch size, dec hid dim]
        
        #seq len, n layers and n directions will always be 1 in this decoder, therefore:
        #output = [1, batch size, dec hid dim]
        #hidden = [1, batch size, dec hid dim]
        #this also means that output == hidden
        
        embedded = embedded.squeeze(0)
        output = output.squeeze(0)
        weighted = weighted.squeeze(0)
        
        prediction = self.fc_out(mnp.concatenate((output, weighted, embedded), axis = 1))
        
        #prediction = [batch size, output dim]
        
        return prediction, hidden.squeeze(0), a.squeeze(1)

### Seq2Seq

In [16]:
import random

class Seq2Seq(nn.Cell):
    def __init__(self, encoder, decoder, src_pad_idx, teacher_forcing_ration):
        super().__init__()
        
        self.encoder = encoder
        self.decoder = decoder
        self.src_pad_idx = src_pad_idx
        self.teacher_forcing_ratio = teacher_forcing_ration
        self.random = ops.UniformReal()
        
    def create_mask(self, src):
        mask = (src != self.src_pad_idx).astype(mindspore.int32).swapaxes(1, 0)
        return mask
        
    def construct(self, src, src_len, trg, trg_len=None):
        #src = [src len, batch size]
        #src_len = [batch size]
        #trg = [trg len, batch size]
        #teacher_forcing_ratio is probability to use teacher forcing
        #e.g. if teacher_forcing_ratio is 0.75 we use teacher forcing 75% of the time
        if trg_len is None:
            trg_len = trg.shape[0]
        
        #tensor to store decoder outputs
        outputs = []
        
        #encoder_outputs is all hidden states of the input sequence, back and forwards
        #hidden is the final forward and backward hidden states, passed through a linear layer
        encoder_outputs, hidden = self.encoder(src, src_len)
                
        #first input to the decoder is the <sos> tokens
        inputs = trg[0]
        
        mask = self.create_mask(src)

        #mask = [batch size, src len]
                
        for t in range(1, trg_len):
            
            #insert input token embedding, previous hidden state, all encoder hidden states 
            #  and mask
            #receive output tensor (predictions) and new hidden state
            output, hidden, _ = self.decoder(inputs, hidden, encoder_outputs, mask)
            # print(output)
            #place predictions in a tensor holding predictions for each token
            outputs.append(output)
            
            #get the highest predicted token from our predictions
            top1 = output.argmax(1) 

            if self.training:
                #decide if we are going to use teacher forcing or not
                teacher_force = self.random((1,)) < self.teacher_forcing_ratio
                # teacher_force = random.random() < self.teacher_forcing_ratio
                #if teacher forcing, use actual next token as next input
                #if not, use predicted token
                inputs = trg[t] if teacher_force else top1
            else:
                inputs = top1
        
        outputs = mnp.stack(outputs, axis=0)
            
        return outputs.astype(dtype)

## 模型训练

模型参数，编码器，注意力层，解码器以及seq2seq网络初始化。

In [17]:
input_dim = len(de_vocab) # 输入维度
output_dim = len(en_vocab) # 输出维度
enc_emb_dim = 256 # Encoder Embedding层维度
dec_emb_dim = 256 # Decoder Embedding层维度
enc_hid_dim = 512 # Encoder 隐藏层维度
dec_hid_dim = 512 # Decoder 隐藏层维度
enc_dropout = 0.5 # Encoder Dropout
dec_dropout = 0.5 # Decoder Dropout
src_pad_idx = de_vocab.pad_idx # 德语词典中pad占位符的数字索引
trg_pad_idx = en_vocab.pad_idx # 英语词典中pad占位符的数字索引

compute_dtype = mindspore.float32 # 计算中数据的类型
dtype = mindspore.float32 # 返回数据的类型

attn = Attention(enc_hid_dim, dec_hid_dim)
encoder = Encoder(input_dim, enc_emb_dim, enc_hid_dim, dec_hid_dim, enc_dropout)
decoder = Decoder(output_dim, dec_emb_dim, enc_hid_dim, dec_hid_dim, dec_dropout, attn)

model = Seq2Seq(encoder, decoder, src_pad_idx, 0.5)

损失函数、优化器初始化。

In [18]:
opt = nn.Adam(model.trainable_params(), learning_rate=0.001) # 损失函数
loss_fn = nn.CrossEntropyLoss(ignore_index=trg_pad_idx) # 优化器

注意在模型训练中，可能会出现权重更新过大的情况。这会导致数值上溢或者下溢，最终造成梯度爆炸（gradient explosion）。为解决这个问题，我们需要在反向传播计算梯度之后，使用梯度裁剪(gradient clipping)，再将裁剪后的梯度传入优化器进行网络更新。

In [ ]:
# def forward_fn(model, src, src_len, trg):
#     """前向网络"""
#     src = src.swapaxes(0, 1)
#     trg = trg.swapaxes(0, 1)

#     output = model(src, src_len, trg)
#     output_dim = output.shape[-1]
#     output = output.view(-1, output_dim)
#     trg = trg[1:].view(-1)
#     loss = loss_function(output, trg)

#     return loss

# # 反向传播计算梯度
# grad_fn = ops.value_and_grad(forward_fn, None, opt.parameters)
# # grad_fn = mindspore.value_and_grad(forward_fn, None, opt.parameters)

# def train_step(model, src, src_len, trg, clip):
#     """单步训练"""
#     loss, grads = grad_fn(model, src, src_len, trg)
#     grads = ops.clip_by_global_norm(grads, clip_norm=clip)  # 梯度裁剪
#     optimizer(grads)  # 更新网络参数

#     return loss

In [ ]:
# def train(model, iterator, optimizer, loss_function, clip, epoch=0):
#     print('[Training Loop]')
#     model.set_train(True)
#     num_batches = len(iterator)
#     total_loss = 0
#     total_steps = 0

#     with tqdm(total=num_batches) as t:
#         t.set_description(f'Epoch: {epoch}')
#         for src, src_len, trg in iterator():
#             loss = train_step(model, src, src_len, trg, clip)
#             total_loss += loss.asnumpy()
#             total_steps += 1
#             curr_loss = total_loss / total_steps
#             t.set_postfix({'loss': f'{curr_loss:.2f}'})
#             t.update(1)
    
#     return total_loss / total_steps

In [ ]:
# def evaluate(model, iterator, loss_function):
#     print('[Validation Loop]')
#     model.set_train(False)
#     num_batches = len(iterator)
#     total_loss = 0
#     total_steps = 0
    
#     with tqdm(total=num_batches) as t:
#         for src, src_len, trg in iterator():
#             loss = forward_fn(model, src, src_len, trg)
#             total_loss += loss.asnumpy()
#             total_steps += 1
#             curr_loss = total_loss / total_steps
#             t.set_postfix({'loss': f'{curr_loss:.2f}'})
#             t.update(1)
    
#     return total_loss / total_steps

In [22]:
def train(model, iterator, optimizer, loss_function, clip, epoch=0):
    print('[Training Loop]')
    model.set_train(True)
    num_batches = len(iterator)
    total_loss = 0
    total_steps = 0

    def forward_fn(src, src_len, trg):
        """前向网络"""
        src = src.swapaxes(0, 1)
        trg = trg.swapaxes(0, 1)

        output = model(src, src_len, trg)
        output_dim = output.shape[-1]
        output = output.view(-1, output_dim)
        trg = trg[1:].view(-1)
        loss = loss_function(output, trg)

        return loss

    # 反向传播计算梯度
    grad_fn = ops.value_and_grad(forward_fn, None, opt.parameters)
    # grad_fn = mindspore.value_and_grad(forward_fn, None, opt.parameters)

    def train_step(src, src_len, trg):
        """单步训练"""
        loss, grads = grad_fn(src, src_len, trg)
        grads = ops.clip_by_global_norm(grads, clip_norm=clip)  # 梯度裁剪
        optimizer(grads)  # 更新网络参数

        return loss

    with tqdm(total=num_batches) as t:
        t.set_description(f'Epoch: {epoch}')
        for src, src_len, trg in iterator():
            loss = train_step(src, src_len, trg)
            total_loss += loss.asnumpy()
            total_steps += 1
            curr_loss = total_loss / total_steps
            t.set_postfix({'loss': f'{curr_loss:.2f}'})
            t.update(1)
    
    return total_loss / total_steps

在训练中使用验证数据集对模型进行验证，保存效果最好的模型。

In [23]:
def evaluate(model, iterator, loss_function):
    print('[Validation Loop]')
    model.set_train(False)
    num_batches = len(iterator)
    total_loss = 0
    total_steps = 0
    
    def forward_fn(src, src_len, trg):
        """前向网络"""
        src = src.swapaxes(0, 1)
        trg = trg.swapaxes(0, 1)

        output = model(src, src_len, trg)
        output_dim = output.shape[-1]
        output = output.view(-1, output_dim)
        trg = trg[1:].view(-1)
        loss = loss_function(output, trg)

        return loss
    
    with tqdm(total=num_batches) as t:
        for src, src_len, trg in iterator():
            loss = forward_fn(src, src_len, trg)
            total_loss += loss.asnumpy()
            total_steps += 1
            curr_loss = total_loss / total_steps
            t.set_postfix({'loss': f'{curr_loss:.2f}'})
            t.update(1)
    
    return total_loss / total_steps

In [24]:
from mindspore import save_checkpoint

num_epochs = 10
clip = 1.0
best_valid_loss = float('inf')
ckpt_file_name = os.path.join(cache_dir, 'seq2seq.ckpt')
mindspore.set_context(mode=mindspore.PYNATIVE_MODE)

for i in range(num_epochs):
    train_loss = train(model, train_iterator, opt, loss_fn, clip, i)
    valid_loss = evaluate(model, valid_iterator, loss_fn)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        save_checkpoint(model, ckpt_file_name)

[Training Loop]


Epoch: 0: 100%|███████████████████████████████████████████████████████████| 226/226 [34:23<00:00,  9.13s/it, loss=5.05]


[Validation Loop]


100%|█████████████████████████████████████████████████████████████████████████| 8/8 [00:15<00:00,  1.97s/it, loss=4.57]


[Training Loop]


...

100%|█████████████████████████████████████████████████████████████████████████| 8/8 [00:15<00:00,  1.96s/it, loss=3.19]


[Training Loop]


Epoch: 9: 100%|███████████████████████████████████████████████████████████| 226/226 [35:17<00:00,  9.37s/it, loss=1.66]


[Validation Loop]


100%|█████████████████████████████████████████████████████████████████████████| 8/8 [00:15<00:00,  1.90s/it, loss=3.32]


## 模型推理

In [25]:
def translate_sentence(sentence, de_vocab, en_vocab, model, max_len=32):
    """给定德语句子，返回英文翻译"""
    model.set_train(False)
    # 对输入句子进行分词
    if isinstance(sentence, str):
        spacy_lang = spacy.load('de')
        tokens = [token.text.lower() for token in spacy_lang(sentence)]
    else:
        tokens = [token.lower() for token in sentence]
    
    # 补充起始、终止占位符，统一序列长度
    if len(tokens) > max_len - 2:
        src_len = max_len
        tokens = ['<bos>'] + tokens[:max_len - 2] + ['<eos>']
    else:
        src_len = len(tokens) + 2
        tokens = ['<bos>'] + tokens + ['<eos>'] + ['<pad>'] * (max_len - src_len)
        
    # 将德语单词转化为数字索引
    src = de_vocab.encode(tokens)
    src = mindspore.Tensor(src, mindspore.int32).expand_dims(1)
    src_len = mindspore.Tensor([src_len], mindspore.int32)
    trg = mindspore.Tensor([en_vocab.bos_idx], mindspore.int32).expand_dims(1)
    
    # 获得预测结果，并将其转化为英语单词
    outputs = model(src, src_len, trg, max_len)
    trg_indexes = [int(i.argmax(1).asnumpy()) for i in outputs]
    eos_idx = trg_indexes.index(en_vocab.eos_idx) if en_vocab.eos_idx in trg_indexes else -1
    trg_tokens = en_vocab.decode(trg_indexes[:eos_idx])
    
    return trg_tokens

加载之前训练好的模型。

In [26]:
from mindspore import load_checkpoint, load_param_into_net

param_dict = load_checkpoint(ckpt_file_name)
load_param_into_net(model, param_dict)

[]

使用测试数据集中的任意一组文本数据进行预测。

In [27]:
example_idx = 0

src = test_dataset[example_idx][0]
trg = test_dataset[example_idx][1]

print(f'src = {src}')
print(f'trg = {trg}')

src = ['ein', 'mann', 'mit', 'einem', 'orangefarbenen', 'hut', ',', 'der', 'etwas', 'anstarrt', '.']
trg = ['a', 'man', 'in', 'an', 'orange', 'hat', 'starring', 'at', 'something', '.']


查看预测结果。

In [28]:
translation = translate_sentence(src, de_vocab, en_vocab, model)

print(f'predicted trg = {translation}')

predicted trg = ['a', 'man', 'with', 'an', 'orange', 'hat', 'hat', 'is', 'something', '.', '.']


## BLEU得分

双语替换评测得分（bilingual evaluation understudy，BLEU）为衡量文本翻译模型生成出来的语句好坏的一种算法，它的核心在于评估机器翻译的译文 $\text{pred}$ 与人工翻译的参考译文 $\text{label}$ 的相似度。通过对机器译文的片段与参考译文进行比较，计算出各个片段的的分数，并配以权重进行加和，基本规则为：

1. 惩罚过短的预测，即如果机器翻译出来的译文相对于人工翻译的参考译文过于短小，则命中率越高，需要施加更多的惩罚；
2. 对长段落匹配更高的权重，即如果出现长段落的完全命中，说明机器翻译的译文更贴近人工翻译的参考译文；

BLEU的公式如下：

$$exp(min(0, 1-\frac{len(\text{label})}{len(\text{pred})})\Pi^k_{n=1}p_n^{1/2^n})$$

- `len(label)`：人工翻译的译文长度
- `len(pred)`：机器翻译的译文长度
- `p_n`：n-gram的精度

我们可以调用`nltk`中的`corpus_bleu`函数来计算BLEU，在此之前，需要手动下载`nltk`。
> pip install nltk

In [29]:
from nltk.translate.bleu_score import corpus_bleu

def calculate_bleu(dataset, de_vocab, en_vocab, model, max_len=50):
    trgs = []
    pred_trgs = []
    
    for data in dataset:
        
        src = data[0]
        trg = data[1]
        
        pred_trg = translate_sentence(src, de_vocab, en_vocab, model, max_len)
                
        pred_trgs.append(pred_trg)
        trgs.append([trg])
        
    return corpus_bleu(trgs, pred_trgs)

In [30]:
bleu_score = calculate_bleu(test_dataset, de_vocab, en_vocab, model)

print(f'BLEU score = {bleu_score*100:.2f}')

BLEU score = 19.96


In [33]:
# 和pytorch测试集的loss对比

test_loss = evaluate(model, test_iterator, loss_fn)

[Validation Loop]


100%|█████████████████████████████████████████████████████████████████████████| 8/8 [00:12<00:00,  1.51s/it, loss=3.13]
